In [ ]:
from ipywidgets import VBox, HBox, jslink
import ipyvuetify as v
from typing import List, Tuple
import uuid

# Background "database"
name_database = [
    ("Meikäläinen", "Matti", uuid.uuid4()),
    ("Meikäläinen", "Maija", uuid.uuid4()),
    ("Autoilija", "Meri Anna Aurora", uuid.uuid4()),
    ("Rotko", "Voima Otto Akseli", uuid.uuid4()),
]


# BUSINESS LOGIC


def filter(data, query):
    _data = data.copy()

    for i, (k, ignore, ignore) in sorted(enumerate(data), reverse=True):
        if query.casefold() not in k.casefold():
            del _data[i]

    return _data


def create(data, last_name, first_name):
    _data = data.copy()
    _data.append((last_name, first_name, uuid.uuid4()))

    return _data


def update(data, uuid, last_name, first_name):
    _data = data.copy()
    for item in _data:
        if item[2] == uuid:
            _data[_data.index(item)] = (last_name, first_name, uuid)

    return _data


def delete(data, uuid):
    _data = data.copy()

    for item in _data:
        if item[2] == uuid:
            del _data[_data.index(item)]

    return _data


# UI Layer


def update_data(new_data):
    global name_database
    name_database = new_data
    update_list()


def find_uuid(selected_index: int, query: str):
    _list = filter(name_database, query)

    return _list[selected_index][2]


def update_list(query: str = ""):
    _names = filter(name_database, query)
    selected_row = None
    selector.v_model = selected_row

    selector.children = [
        v.ListItem(
            children=[
                v.ListItemAvatar(children=[v.Icon(children=["mdi-account"])]),
                v.ListItemContent(
                    children=[
                        v.ListItemTitle(children=[last]),
                        v.ListItemSubtitle(children=[first]),
                    ]
                ),
            ]
        )
        for last, first, ignore in _names
    ]


def select(i):
    global selected_row
    selected_row = i["new"]
    Buttons[1].disabled = i["new"] is None
    Buttons[2].disabled = i["new"] is None


filter_field = v.TextField(v_model="", placeholder="Filter", description="string")

selector = v.ListItemGroup(v_model=selected_row, color="primary")
update_list()
container_children = [v.List(children=[selector], two_line=True)]
list_container = v.Container(children=container_children, style_="overflow-y: scroll;")

last_field = v.TextField(
    v_model="", placeholder="Last Name", style_="margin-left: 20px;"
)
first_field = v.TextField(
    v_model="", placeholder="First Name", style_="margin-left: 20px;"
)

Buttons = [
    v.Btn(children="Create", style_="margin: 0 10px;"),
    v.Btn(children="Update", disabled=selected_row is None, style_="margin: 0 10px;"),
    v.Btn(children="Delete", disabled=selected_row is None, style_="margin: 0 10px;"),
]


Buttons[0].on_event(
    "click",
    lambda *ignore_args: update_data(
        create(name_database, last_field.v_model, first_field.v_model)
    ),
)
Buttons[1].on_event(
    "click",
    lambda *ignore_args: update_data(
        update(
            name_database,
            find_uuid(selected_row, filter_field.v_model),
            last_field.v_model,
            first_field.v_model,
        )
    ),
)
Buttons[2].on_event(
    "click",
    lambda *ignore_args: update_data(
        delete(name_database, find_uuid(selected_row, filter_field.v_model))
    ),
)
filter_field.observe(lambda change: update_list(change["new"]), "v_model")
selector.observe(select, "v_model")


page = [
    filter_field,
    v.Container(
        children=[list_container, VBox([last_field, first_field])],
        style_="display: flex; width: 100%; max-height: 300px; flex-grow: 1 0;",
    ),
    HBox([*Buttons]),
]


VBox(page)